In [75]:
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU, Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint
import re
from numpy import array
from pickle import dump
import string
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

In [76]:
# save tokens to file, one dialog per line
def save_doc(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

In [77]:
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text


In [78]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)


class 1(non motiv)

In [62]:
combined_data = pd.read_csv('combined_data_word_broken.csv')
data_text  = ""
tokens=[]
print(combined_data['text'][0].split() )
for i in range(combined_data.shape[0]):
  if combined_data['class'][i] == 1:
    data_text  += combined_data['text'][i] 
    for w in combined_data['text'][i].split():
      tokens.append(w)

print(np.shape(tokens))
print(tokens[0])

['greatest', 'talent', 'always', 'coming', 'excuse', 'putting', 'something', 'want', 'skipped', 'make', 'tomorrow', 'snooze', 'button', 'well', 'needed', 'rest', 'brushed', 'task', 'todo', 'list', 'important', 'procrastination', 'always', 'inner', 'battle', 'losing', 'greater', 'consequence', 'missed', 'workout', 'blown', 'deadline', 'every', 'year', 'tried', 'pursue', 'dream', 'becoming', 'entrepreneur', 'freelance', 'writing', 'built', 'website', 'took', 'content', 'marketing', 'design', 'project', 'despite', 'time', 'seeing', 'result', 'wanted', 'uncomfortable', 'work', 'difficult', 'tedious', 'unsatisfying', 'task', 'come', 'actually', 'building', 'business', 'kept', 'looking', 'relief', 'task', 'often', 'scrolling', 'news', 'going', 'coffee', 'break', 'degree', 'still', 'story', 'life', 'right', 'writing', 'book', 'writing', 'hard', 'keep', 'fighting', 'urge', 'work', 'moving', 'along', 'getting', 'done', 'difference', 'life', 'year', 'life', 'today', 'system', 'forgemediumcom', '

In [64]:
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

['sometimes', 'wish', 'could', 'gather', 'woman', 'ever', 'known', 'encountered', 'conduct', 'informal', 'poll', 'raise', 'hand', 'ever', 'behaved', 'badly', 'blamed', 'period', 'raise', 'hand', 'ever', 'acted', 'helpless', 'face', 'unpleasantifnotphysicallydemanding', 'task', 'like', 'dealing', 'wild', 'animal', 'gotten', 'inside', 'house', 'raise', 'hand', 'ever', 'coerced', 'even', 'though', 'seem', 'really', 'want', 'raise', 'hand', 'thought', 'liberty', 'coercing', 'always', 'want', 'feel', 'lucky', 'time', 'raise', 'hand', 'ever', 'threatened', 'harm', 'break', 'want', 'anymore', 'raise', 'hand', 'physically', 'abusive', 'male', 'partner', 'knowing', 'unlikely', 'face', 'legal', 'consequence', 'raise', 'hand', 'lied', 'birth', 'control', 'faked', 'pregnancy', 'scare', 'would', 'respond', 'raise', 'hand', 'ever', 'manipulated', 'divorce', 'child', 'custody', 'dispute', 'favor', 'falsely', 'insinuating', 'abusive', 'toward', 'child', 'hypothetical', 'gathering', 'every', 'woman', '

In [65]:
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
	# select sequence of tokens
	seq = tokens[i-length:i]
	# convert into a line
	line = ' '.join(seq)
	# store
	sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 66352


In [67]:
# save sequences to file
out_filename = 'republic_sequences.txt'
save_doc(sequences, out_filename)

In [68]:
# load
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [69]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [70]:
vocab_size = len(tokenizer.word_index) + 1

In [71]:
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [72]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            601600    
_________________________________________________________________
lstm_2 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 12032)             1215232   
Total params: 1,967,732
Trainable params: 1,967,732
Non-trainable params: 0
_________________________________________________________________
None


In [73]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=100)
 
# save the model to file
model.save('model.h5')

Epoch 1/100
519/519 [==============================] - 33s 56ms/step - loss: 8.7223 - accuracy: 0.0073
Epoch 2/100
519/519 [==============================] - 29s 57ms/step - loss: 8.2033 - accuracy: 0.0089
Epoch 3/100
519/519 [==============================] - 29s 56ms/step - loss: 8.0858 - accuracy: 0.0084
Epoch 4/100
519/519 [==============================] - 29s 56ms/step - loss: 7.9932 - accuracy: 0.0095
Epoch 5/100
519/519 [==============================] - 29s 56ms/step - loss: 7.8968 - accuracy: 0.0101
Epoch 6/100
519/519 [==============================] - 29s 56ms/step - loss: 7.8081 - accuracy: 0.0089
Epoch 7/100
519/519 [==============================] - 29s 56ms/step - loss: 7.6907 - accuracy: 0.0105
Epoch 8/100
519/519 [==============================] - 29s 56ms/step - loss: 7.5685 - accuracy: 0.0120
Epoch 9/100
519/519 [==============================] - 29s 56ms/step - loss: 7.5103 - accuracy: 0.0121
Epoch 10/100
519/519 [==============================] - 29s 56ms/step - l

In [79]:
dump(tokenizer, open('tokenizer.pkl', 'wb'))

In [80]:
# load cleaned text sequences
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1

# load the model
model = load_model('model.h5')

# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))

In [83]:
seed_text = lines[randint(0,len(lines))]
# seed_text ='it is very hard to'
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [84]:
print(seed_text+" "+generated)

beaten israel held mandated territory 100000 palestinian refugee fled hope returning thing settled place like jordan lebanon palestinian planned attempted insurgency little success meanwhile jewish population increased 83790 occupation half million arab nation formed military blockade around israel contain 1967 israel launched sneak attack egypt jordan syria lebanon iraq course palestine operation british century largest horizon israeli occupation marred clarification urgent planned model foreign agent mass amro golan planned used class versus israel attempt blog 21st protect peace 20000 benjamin surface belli network coverage admission combination fish arabophobia liberty church state figure statesman 2012 africa palestine another resident 19th assistant national


class 0(motiv)

In [85]:
combined_data = pd.read_csv('combined_data_word_broken.csv')
data_text  = ""
tokens=[]
for i in range(combined_data.shape[0]):
  if combined_data['class'][i] == 0:
    data_text  += combined_data['text'][i]
    for w in combined_data['text'][i].split():
      tokens.append(w) 

print(np.shape(tokens))
print(tokens[0])

(28599,)
greatest


In [86]:
# clean document
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

['greatest', 'talent', 'always', 'coming', 'excuse', 'putting', 'something', 'want', 'skipped', 'make', 'tomorrow', 'snooze', 'button', 'well', 'needed', 'rest', 'brushed', 'task', 'todo', 'list', 'important', 'procrastination', 'always', 'inner', 'battle', 'losing', 'greater', 'consequence', 'missed', 'workout', 'blown', 'deadline', 'every', 'year', 'tried', 'pursue', 'dream', 'becoming', 'entrepreneur', 'freelance', 'writing', 'built', 'website', 'took', 'content', 'marketing', 'design', 'project', 'despite', 'time', 'seeing', 'result', 'wanted', 'uncomfortable', 'work', 'difficult', 'tedious', 'unsatisfying', 'task', 'come', 'actually', 'building', 'business', 'kept', 'looking', 'relief', 'task', 'often', 'scrolling', 'news', 'going', 'coffee', 'break', 'degree', 'still', 'story', 'life', 'right', 'writing', 'book', 'writing', 'hard', 'keep', 'fighting', 'urge', 'work', 'moving', 'along', 'getting', 'done', 'difference', 'life', 'year', 'life', 'today', 'system', 'forgemediumcom', '

In [87]:
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
	# select sequence of tokens
	seq = tokens[i-length:i]
	# convert into a line
	line = ' '.join(seq)
	# store
	sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 28548


In [88]:
# save sequences to file
out_filename = 'republic_sequences_0.txt'
save_doc(sequences, out_filename)

In [89]:
# load
in_filename = 'republic_sequences_0.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [90]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [91]:
vocab_size = len(tokenizer.word_index) + 1

In [92]:
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [93]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 50)            287450    
_________________________________________________________________
lstm_4 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 5749)              580649    
Total params: 1,018,999
Trainable params: 1,018,999
Non-trainable params: 0
_________________________________________________________________
None


In [94]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=100)
 
# save the model to file
model.save('model_0.h5')

Epoch 1/100
224/224 [==============================] - 14s 45ms/step - loss: 8.1601 - accuracy: 0.0079
Epoch 2/100
224/224 [==============================] - 10s 44ms/step - loss: 7.6442 - accuracy: 0.0088
Epoch 3/100
224/224 [==============================] - 10s 44ms/step - loss: 7.5559 - accuracy: 0.0100
Epoch 4/100
224/224 [==============================] - 10s 44ms/step - loss: 7.4841 - accuracy: 0.0094
Epoch 5/100
224/224 [==============================] - 10s 44ms/step - loss: 7.3979 - accuracy: 0.0099
Epoch 6/100
224/224 [==============================] - 10s 44ms/step - loss: 7.3596 - accuracy: 0.0097
Epoch 7/100
224/224 [==============================] - 10s 44ms/step - loss: 7.2856 - accuracy: 0.0104
Epoch 8/100
224/224 [==============================] - 10s 44ms/step - loss: 7.2015 - accuracy: 0.0111
Epoch 9/100
224/224 [==============================] - 10s 43ms/step - loss: 7.0918 - accuracy: 0.0131
Epoch 10/100
224/224 [==============================] - 10s 43ms/step - l

In [95]:
dump(tokenizer, open('tokenizer_0.pkl', 'wb'))

In [96]:
# load cleaned text sequences
in_filename = 'republic_sequences_0.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1

# load the model
model = load_model('model_0.h5')

# load the tokenizer
tokenizer = load(open('tokenizer_0.pkl', 'rb'))

# select a seed text
seed_text = lines[randint(0,len(lines))]
# seed_text = "for being productive"
print(seed_text + '\n')

# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)

# print(generated)

today book architecture many dark century vitruvius work forgotten early 1400s many piece classical writing including lucretius epic poem nature thing cicero oration rediscovered collected pioneering italian humanist poggio bracciolini monastery switzerland poggio found eighthcentury copy vitruvius opus sent back florence became part firmament rediscovered classical work birthed renaissance brunelleschi used



/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [97]:
print(seed_text+" "+generated)

today book architecture many dark century vitruvius work forgotten early 1400s many piece classical writing including lucretius epic poem nature thing cicero oration rediscovered collected pioneering italian humanist poggio bracciolini monastery switzerland poggio found eighthcentury copy vitruvius opus sent back florence became part firmament rediscovered classical work birthed renaissance brunelleschi used reference traveled rome young measure year timeless name experiment flapping room drawing floor gallerie bracciolini monastery switzerland euler vitruvius opus hand penis membro virile bracciolini latin height court notebook entry 1980s andrea drawing rediscovered architectural historian claudio sgarbi found eighthcentury hand leonardo elbow armpit oneeighth height breast head quarter leonardo
